## e2eET Skeleton Based HGR Using Data-Level Fusion

In [39]:
import pickle
import warnings
import numpy as np
from pathlib import Path
from scipy import ndimage
from sklearn.model_selection import train_test_split

---

In [40]:
REORDERING_INDICES = np.array([0, 1, 6, 7, 8, 2, 9, 10, 11, 3, 12, 13, 14, 4, 15, 16, 17, 5, 18, 19, 20])

FPHA_CAM_EXTRINSICS = np.array(
    [
        [0.999988496304, -0.00468848412856, 0.000982563360594, 25.7],
        [0.00469115935266, 0.999985218048, -0.00273845880292, 1.22],
        [-0.000969709653873, 0.00274303671904, 0.99999576807, 3.902],
        [0, 0, 0, 1],
    ]
)

FPHA_CAM_INTRINSICS = np.array(
    [
        [1395.749023, 0, 935.732544],
        [0, 1395.749268, 540.681030],
        [0, 0, 1],
    ]
)

In [41]:
def _resize_gestures(in_gest_seqs, target_length=250):
    """Resize the time series by interpolating them to the same length"""

    out_gest_seqs = []
    for sequence in in_gest_seqs:
        zoomed_skeletons = []
        for skeleton in range(np.size(sequence, 1)):
            _zoom_skel = ndimage.zoom(sequence.T[skeleton], target_length / len(sequence), mode="reflect")
            zoomed_skeletons.append(_zoom_skel)

        out_gest_seqs.append(np.array(zoomed_skeletons).T)

    return np.array(out_gest_seqs)

In [42]:
def load_txt_gestures(type, resize_length, root, verbose=False):
    """
    Get the 3D pose gestures sequences, and their associated labels.
    Output:  a tuple of (gestures, labels, subjects).
    """
    
    # The following files are empty:
    #     [1] ./FPHA/Hand_Pose_Annotation_v1/Subject_2/close_milk/4/skeleton.txt
    #     [2] ./FPHA/Hand_Pose_Annotation_v1/Subject_2/put_tea_bag/2/skeleton.txt
    #     [3] ./FPHA/Hand_Pose_Annotation_v1/Subject_4/flip_sponge/2/skeleton.txt
    # The following files are missing:
    #     [1] ./FPHA/Hand_Pose_Annotation_v1/Subject_2/charge_cell_phone/2/skeleton.txt

    # _____
    assert "FPHA" in root, "Check that the correct dataset folder is provided!"
    assert type in ["2d", "3d"], "Data type has to be specified ['2d' / '3d']"

    root = Path(f"{root}/Hand_Pose_Annotation_v1")
    filenames = list(root.rglob("*skeleton.txt"))
    
    # _____
    gestures = []
    labels = []
    subjects = []

    for f in filenames:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            _gesture = np.genfromtxt(f, dtype=np.float32)

        if verbose: print(f"{f= } | {_gesture.shape= }")
        if not _gesture.tolist():
            print(f"Warning: empty file @'{str(f)}'; Skipping...")
            continue

        # apply transformations; these operations are taken from the source code
        _transformed_gesture = []
        _gesture = _gesture[:, 1:].reshape(-1, 21, 3)
        
        for _skel in _gesture:
            # reorder indices
            _skel = _skel[REORDERING_INDICES]

            # apply camera extrinsic to hand skeleton; for 3d data
            skel_hom3d = np.concatenate([_skel, np.ones([_skel.shape[0], 1])], 1)
            skel_camcoords = FPHA_CAM_EXTRINSICS.dot(skel_hom3d.transpose()).transpose()[:, :3]
            skel_camcoords = skel_camcoords.astype(np.float32)

            if type == "3d":
                _transformed_gesture.append(skel_camcoords)

            elif type == "2d":
                # apply camera intrinsic to hand skeleton; for 2d data
                skel_hom2d = np.array(FPHA_CAM_INTRINSICS).dot(skel_camcoords.transpose()).transpose()
                skel_proj = (skel_hom2d / skel_hom2d[:, 2:])[:, :2]
                _transformed_gesture.append(skel_proj)

        _gesture = np.array(_transformed_gesture).reshape(_gesture.shape[0], -1)
        gestures.append(_gesture)

        _label = f.parent.parent
        # _subject_label = _label.parent.name
        _label = _label.name.replace("_", " ").title().replace(" ", "")
        labels.append(_label)

        # _subject_label += f"_{_label}"
        _subject_label = str(f.parent.relative_to(root)).replace('\\', '/')
        subjects.append(_subject_label)

        if verbose: print(f"{_gesture.shape= } | {_label= } | {_subject_label= }")
        # break

    if resize_length: gestures = _resize_gestures(gestures, target_length=resize_length)
    assert len(gestures) == len(labels) == len(subjects)
    return gestures, labels, subjects

# _ = load_txt_gestures(type="3d", resize_length=None, root="../../FPHA")

In [43]:
def _write_data(data, filepath):
    """Save the dataset to a file. Note: data is a dict with keys 'X_train', ..."""

    with open(filepath, "wb") as output_file: pickle.dump(data, output_file)

In [44]:
def load_pckl_data(filepath):
    """
    Returns hand gesture sequences (X) and their associated labels (Y).
    """

    file = open(filepath, "rb")
    data = pickle.load(file, encoding="latin1") # change to 'latin1' to 'utf8' if the data does not load
    file.close()

    return (
        data["X_train"], data["X_valid"],
        data["train_labels"], data["valid_labels"],
        data["train_subjects"], data["valid_subjects"],
    )

In [55]:
def create_FPHA_paper_split(type, root, resize_length=None, seed=17711, save_path=None):
    assert type in ["2d", "3d"], "Data type has to be specified ['2d' / '3d']"
    
    # load the dataset gesture sequences from file(s)
    gestures, labels, subjects = load_txt_gestures(type, resize_length, root)
    print(">>> <gestures, labels, subjects> loaded successfully!")

    # load the paper data splits from file
    with open(f"{root}/data_split_action_recognition.txt", "r") as f:
        splits = [l.split()[0] for l in f.readlines()]
    
    # split into train and validation subsets 
    train_split, valid_split = splits[1:601], splits[602:]
    idxs_valid = [i for i, i_subject in enumerate(subjects) if i_subject in valid_split]
    idxs_train = [i for i, i_subject in enumerate(subjects) if i_subject in train_split]

    X_train, X_valid = gestures[idxs_train], gestures[idxs_valid]  # type:ignore
    train_labels = np.array(labels)[idxs_train].tolist()
    valid_labels = np.array(labels)[idxs_valid].tolist()
    train_subjects = np.array(subjects)[idxs_train].tolist()
    valid_subjects = np.array(subjects)[idxs_valid].tolist()
    print(f">>> {type} training ({X_train.shape}) and validation ({X_valid.shape}) data created.")
    
    # save the test-train data to disk
    if save_path is None: save_path = "../datasets"
    save_path = f"{save_path}/FPHA_{type}_dictPaperSplit_l{resize_length}_s{len(gestures)}.pckl"
        
    data = {
        "X_train": X_train, "X_valid": X_valid,
        "train_labels": train_labels, "valid_labels": valid_labels,
        "train_subjects": train_subjects, "valid_subjects": valid_subjects,
    }
    _write_data(data, filepath=save_path)
    print(f">>> FPHA paper train-valid data written to <{save_path}> successfully!")

---

In [56]:
create_FPHA_paper_split(type="3d", root="../datasets/FPHA", resize_length=250)

>>> <gestures, labels, subjects> loaded successfully!
>>> 3d training ((600, 250, 63)) and validation ((575, 250, 63)) data created.
>>> FPHA paper train-valid data written to <../datasets/FPHA_3d_dictPaperSplit_l250_s1175.pckl> successfully!
